In [1]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

2019-08-08 16:02:25,189 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-08-08 16:02:25,220 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating administrator mocks.
2019-08-08 16:02:25,260 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating administrator mocks.
2019-08-08 16:02:25,261 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-08-08 16:02:25,264 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-08-08 16:02:25,265 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-08-08 16:02:25,271 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-08-08 16:02:25,272 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-08-08 16:02:2

In [4]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "enrich_cancel_before_active" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
# Note: this key is case sensitive!!
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation" # if None, input_branch is automagically set to your working branch

In [5]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

2019-08-08 16:03:20,371 - core.logging - DEBUG - Adding/getting mocks for specified configurations...
2019-08-08 16:03:20,397 - core.logging - DEBUG - Done. Creating mock run event and committing results to configuration mocker.


In [6]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[transform name here]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [37]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''
    trans_id: str # Column name to use for Transaction ID
    brand_col: str # Column name to use for Brand
    patient_id: str # Column name to use for Patient ID
    pharmacy: str # Column name to use for Pharmacy
    status_date: str # Column name to use for Status Date
    referral_date: str # Column name to use for Referral Date
    status: str # Column name to use for Status
    substatus: str # Column name to use for Substatus
    hierarchy: str # Column name to use for Hierarchy
    active_status_code: str # Active Shipment Status code, e.g. 'ACTIVE' (customer-specific)
    active_substatus_code: str # Active Shipment Substatus code, e.g. 'SHIPMENT' (customer-specific)
    fulfillment_hierarchy: str # Hierarchy used for statuses after the first fill, e.g. 'ACTIVE - SHIPMENT' (customer-specific)
    remove_from_ttff: str # "Remove from TTFF" Hierarchy designation (customer-specific?)

In [38]:
transform = Transform()

In [39]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

if input_pharma == 'sun':
    transform.trans_id = 'pharm_transaction_id'
    transform.brand_col = 'medication'
    transform.patient_id = 'msa_patient_id'
    transform.pharmacy = 'pharm_code'
    transform.status_date = 'status_date'
    transform.referral_date = 'ref_date'
    transform.status =  'status_code'
    transform.substatus =  'sub_status'
    transform.hierarchy = 'hierarchy'
    datetime = '%Y%m%d'
    transform.active_status_code = 'ACTIVE'
    transform.active_substatus_code = 'SHIPMENT'
    transform.fulfillment_hierarchy = 'FULFILLMENT'
    transform.remove_from_ttff = 'REMOVE FROM TTFF'

else:
    transform.trans_id = 'pmcTransactionId'
    transform.brand_col = 'medication'
    transform.patient_id = 'pmcPatientID'
    transform.pharmacy = 'pharmName'
    transform.status_date = 'statusDate'
    transform.referral_date = 'refDate'
    transform.status =  'statusCode'
    transform.substatus =  'custStatusCode'
    transform.hierarchy = 'hierarchy'
    datetime = '%Y-%b-%d'
    transform.active_status_code = 'SHIPMENT'
    transform.active_substatus_code = 'S01'
    transform.fulfillment_hierarchy = 'FULFILLMENT'
    transform.remove_from_ttff = 'REMOVE FROM TTFF'

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

TTFF Metric prepares data to be utilized by the Spotfire GUI.  The actual TTFF metric is not calculated, but its components are calculated.  Spotfire handles the actual calculation of TTFF, using the output of this transform.

The transform takes all patient journey data up-to and including the first active shipment.  If no active shipment is reported, that entire journey is dropped.
It then calculates the time spent in each of those statuses as (next status_date) - (status_date).  The sum across all statuses for a patient journey would then give the the total TTFF (this part is done in Spotfire).

### Transformation

In [136]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[4]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
df = input_contract.fetch(filters=run_filter)

2019-08-08 17:36:42,040 - core.dataset_contract.DatasetContract - INFO - Fetching dataframe from s3 location s3://ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping.


In [ ]:
### Retrieve current dataset from contract
from core.dataset_diff import DatasetDiff

diff = DatasetDiff(db_transform.id)
df = diff.get_diff(transform_name=transform.input_transform, values=[run_id])

In [29]:
df.shape

(13206, 72)

In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

In [154]:
import numpy as np
import pandas as pd

from pandas.tseries.holiday import USFederalHolidayCalendar as cal

In [12]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

### DATA CLEANING: ADDRESS THIS SECTION BEFORE PIPELINE INTEGRATION

In [137]:
# CLEAN DATA - This step should not be necessary once transform is integrated into pipeline.
#    Extract and map relevant columns
#    Convert dates to datetime format
#    Extract brand from medication
#    Convert substatuses to uppercase
#    Populate null referral dates with the min(status_date) for that patient/pharmacy/brand.
    
def clean_data(cust_input_df, datetime, transform):

    clean_df = (
        cust_input_df
        .loc[:,
             [transform.trans_id,
              transform.patient_id,
              transform.pharmacy,
              transform.brand_col,
              transform.status_date,
              transform.referral_date,
              transform.status,
              transform.substatus]
            ]
        .assign(**{
            transform.status_date : lambda x: (
                pd.to_datetime(
                    x[transform.status_date].str[:8].astype(str),
                    format=datetime,
                    errors='coerce'
                )),
            'min_status_date' : lambda x: (
                x.groupby([transform.patient_id,transform.pharmacy,transform.brand_col])
                [transform.status_date]
                .transform(min)
            )
        })       
        .fillna(value={transform.referral_date:'min_status_date'})
        .assign(**{
            transform.referral_date : lambda x: (
                pd.to_datetime(
                    x[transform.referral_date].str[:8].astype(str),
                    format=datetime,
                    errors='coerce'
                ))
        })
        .dropna()
        .assign(**{
            transform.brand_col : lambda x: (x[transform.brand_col].apply(lambda x: x.split()[0].strip())),
            transform.status : lambda x: (x[transform.status].str.upper()),
            transform.substatus : lambda x: (x[transform.substatus].str.upper())
        })
        .drop(['min_status_date'],axis=1)
        .drop_duplicates()
        .sort_values(
            by=[transform.patient_id, transform.pharmacy, transform.brand_col, transform.status_date, transform.status, transform.trans_id],
            ascending=[True, True, True, True, False, True])
        .reset_index(drop=True)
        .assign(**{transform.hierarchy : lambda x: (np.random.choice(['Dummy', 'Fulfillment', 'Remove from TTFF'], size=len(x)))})
        .assign(**{transform.hierarchy : lambda x: (
            np.where(
                x[transform.substatus] == transform.active_substatus_code,
                transform.fulfillment_hierarchy,
                x[transform.hierarchy]
            )
        )})
    )
    return clean_df

In [138]:
df = clean_data(
    df,
    datetime,
    transform
)

df.head()

,pharm_transaction_id,msa_patient_id,pharm_code,medication,status_date,ref_date,status_code,sub_status,hierarchy
0,182176830,2120001,CVS,ILUMYA,2018-12-31,2018-10-19,ACTIVE,SHIPMENT,FULFILLMENT
1,183711690,2120001,CVS,ILUMYA,2018-12-31,2018-10-19,ACTIVE,SHIPMENT,FULFILLMENT
2,901177673820190325000000,2120006,CVS,ILUMYA,2019-03-25,2019-03-15,PENDING,PATIENT CONTACT,Dummy
3,901177673820190405000000,2120006,CVS,ILUMYA,2019-04-05,2019-03-15,CANCELLED,PATIENT RESPONSE,Fulfillment
4,BRIOVARX_20190104_118503541,2120009,BRV,ILUMYA,2019-01-03,2019-01-03,PENDING,OTHER,Dummy


In [139]:
df.shape

(10094, 9)

### APPLY TRANSFORM LOGIC

In [320]:
# Convert headers to standardized names for schema

trans_id = 'pmcTransactionId'
patient_id = 'pmcPatientId'
pharmacy = 'pharmName'
hierarchy = 'ttffAccruBucket'
status_date_long = 'startDateLong'
next_status_date_long = 'statusDateLong'
min_active_date = 'minShipDateLong'
brand = 'brand'
status_date = 'startDateTs'
next_status_date = 'statusDateTs'
time_in_status = 'timeToFillDays'

output_columns = [
    trans_id,
    patient_id,
    pharmacy,
    hierarchy,
    status_date_long,
    next_status_date_long,
    min_active_date,
    brand,
    status_date,
    next_status_date,
    time_in_status
]

In [321]:
# Map column headers to match TTFF schema standardization
# Assign Patient Journey (pj_id) identifier
# (This ID is used for calculation purposes only.  It will not be published)

def pj(df):
    pj_df = (
        df
        .rename(columns = {
            transform.trans_id: trans_id,
            transform.patient_id: patient_id,
            transform.pharmacy: pharmacy,
            transform.brand_col: brand,
            transform.hierarchy: hierarchy,
            transform.status_date: status_date
        })
        .assign(**{
            'pj_id' : lambda x: (
                x.groupby([patient_id, pharmacy, brand]).grouper.group_info[0]
            )
        })
        .sort_values(
            by=[patient_id, pharmacy, brand, status_date, transform.status, trans_id],
            ascending=[True, True, True, True, False, True])
        .reset_index(drop=True)
    )
    return pj_df

In [335]:
# Time to First Fill calculation:
#    1) Convert all strings to uppercase
#    2) Omit records with "Remove from TTFF" hierarchy
#    3) Filter to only include patient journeys where active shipment occurs
#    4) Get status date of first active shipment (min_active_date) for each patient journey
#    5) Filter to only journeys on or before first active shipment date. And if date = first active shipment date, hierarchy must be fulfillment.
#    6) Calculate time spent in status (only include business days and non-holidays) using np.busday_count
#    7) Convert all dates to long type (calculated as number of seconds between 1/1/1970 and the status date)
#    8) Filter/reorder columns to match schema

def ttff(pj_df):
    ttff_df = (
        pj_df
        .apply(lambda x: (x.str.upper() if x.dtype == 'O' else x))
        .loc[lambda x: (
            (x[hierarchy] != transform.remove_from_ttff)
            &
            (x.pj_id.isin(
                x.loc[
                    (
                        (x[transform.status] == transform.active_status_code)
                        &
                        (x[transform.substatus] == transform.active_substatus_code)
                    ),
                    'pj_id'
                ].tolist()
            ))
        )]
        .assign(**{
            min_active_date : lambda x: (
                x.loc[
                    (x[transform.status] == transform.active_status_code)
                    &
                    (x[transform.substatus] == transform.active_substatus_code)
                ]
                .groupby(['pj_id'])[status_date]
                .transform(min)
            )
        })
        .assign(**{
            min_active_date : lambda x: (
                x.groupby(['pj_id'])[min_active_date]
                .transform(lambda x: x.ffill().bfill())
            )
        })
        .loc[lambda x: (
            (x[status_date] < x[min_active_date])
            |
            (
                (x[status_date] == x[min_active_date])
                &
                (x[hierarchy] == transform.fulfillment_hierarchy)
            )
        )]
        .assign(**{
            next_status_date : lambda x: (
                x.groupby(['pj_id'])[status_date]
                .transform(lambda x: x.shift(-1))
                .fillna(x[status_date])
            ),
            time_in_status : lambda x: (
                np.busday_count(
                    x[status_date].apply(lambda x: x.strftime('%Y-%m-%d')),
                    x[next_status_date].apply(lambda x: x.strftime('%Y-%m-%d')),
                    holidays = pd.to_datetime(cal().holidays()).strftime("%Y-%m-%d").tolist()
                )
            )
        })
        .assign(**{
            status_date_long : lambda x: (
                (x[status_date] - pd.to_datetime('1970-01-01'))
                .dt
                .total_seconds()
                .astype(int)),
            min_active_date : lambda x: (
                (x[min_active_date] - pd.to_datetime('1970-01-01'))
                .dt
                .total_seconds()
                .astype(int)),
            next_status_date_long : lambda x: (
                (x[next_status_date] - pd.to_datetime('1970-01-01'))
                .dt
                .total_seconds()
                .astype(int))
        })
        .loc[:,output_columns]
        .reset_index(drop=True)
    )
    return ttff_df

In [339]:
final_dataframe = (
    df
    .pipe(pj)
    .pipe(ttff)
)

final_dataframe.head()

,pmcTransactionId,pmcPatientId,pharmName,ttffAccruBucket,startDateLong,statusDateLong,minShipDateLong,brand,startDateTs,statusDateTs,timeToFillDays
0,182176830,2120001,CVS,FULFILLMENT,1546214400,1546214400,1546214400,ILUMYA,2018-12-31,2018-12-31,0
1,183711690,2120001,CVS,FULFILLMENT,1546214400,1546214400,1546214400,ILUMYA,2018-12-31,2018-12-31,0
2,11240428,2120026,CGN,DUMMY,1546214400,1546905600,1548201600,ILUMYA,2018-12-31,2019-01-08,5
3,11443176,2120026,CGN,DUMMY,1546905600,1547164800,1548201600,ILUMYA,2019-01-08,2019-01-11,3
4,11551269,2120026,CGN,DUMMY,1547164800,1547510400,1548201600,ILUMYA,2019-01-11,2019-01-15,2


### TEST TRANSFORM OUTPUT

In [342]:
# TEST 1: Check that time in status = 0 for all active shipments

In [ ]:
# TEST 2: Check that there are no "Patient" hierarchies

In [344]:
# TEST 3: Check that next status date is NULL for active shipments
#     Do we really want this? Can we just use current status_date? 
#     Thats how I have it now.  If there are 2 statuses reported on the last day,
#     the sorting is arbitrary, and 1 of the 2 will have same-day and next_status_date, the other would be NA.
#     Might as well be consistent and assign same-day to both.

In [359]:
# TEST 4: Create a "test" dataframe with expected results

test_data = ([
    [1, 0, 'PENDING', 'OTHER', 'PENDING - OTHER'],
    [1, 70, 'PENDING', 'OTHER', 'PENDING - OTHER'],
    [1, 72, 'CANCEL', 'INSURANCE DENIED', 'BVPA'],
    [1, 72, 'CANCEL', 'OTHER', 'BVPA'],
    [1, 72, 'ACTIVE', 'SHIPMENT', transform.fulfillment_hierarchy],
    [2, 0, 'PENDING', 'OTHER', 'REMOVE FROM TTFF'],
    [2, 70, 'PENDING', 'OTHER', 'NO STATUS CLARITY'],
    [2, 72, 'PENDING', 'OTHER', 'PENDING - OTHER'],
    [2, 72, 'CANCEL', 'OTHER', 'PENDING - OTHER'],
    [2, 72, 'ACTIVE', 'READY', transform.fulfillment_hierarchy],
    [2, 73, 'ACTIVE', 'SHIPMENT', transform.fulfillment_hierarchy],
    [2, 73, 'DISCONTINUED', 'OTHER', 'PATIENT'],
    [3, 0, 'PENDING', 'OTHER', 'REMOVE FROM TTFF'],
    [3, 1, 'PENDING', 'OTHER', 'REMOVE FROM TTFF'],
    [3, 2, 'CANCEL', 'OTHER', 'REMOVE FROM TTFF'],
    [3, 70, 'CANCEL', 'OTHER', 'NO STATUS CLARITY'],
    [3, 71, 'ACTIVE', 'READY', transform.fulfillment_hierarchy],
    [3, 72, 'ACTIVE', 'READY', transform.fulfillment_hierarchy]
])

test_df = (
    pd.DataFrame(test_data, columns = [transform.patient_id, transform.status_date, transform.status, transform.substatus, transform.hierarchy])
    .assign(**{
        transform.pharmacy : 'ABC',
        transform.brand_col : 'A',
        transform.status_date : lambda x: (
            pd.to_datetime('2019-01-01', format='%Y-%m-%d') + pd.to_timedelta(x[transform.status_date], unit='d')
        ),
        transform.trans_id : 1
    })
    
)

test_df

,msa_patient_id,status_date,status_code,sub_status,hierarchy,pharm_code,medication,pharm_transaction_id
0,1,2019-01-01,PENDING,OTHER,PENDING - OTHER,ABC,A,1
1,1,2019-03-12,PENDING,OTHER,PENDING - OTHER,ABC,A,1
2,1,2019-03-14,CANCEL,INSURANCE DENIED,BVPA,ABC,A,1
3,1,2019-03-14,CANCEL,OTHER,BVPA,ABC,A,1
4,1,2019-03-14,ACTIVE,SHIPMENT,FULFILLMENT,ABC,A,1
5,2,2019-01-01,PENDING,OTHER,REMOVE FROM TTFF,ABC,A,1
6,2,2019-03-12,PENDING,OTHER,NO STATUS CLARITY,ABC,A,1
7,2,2019-03-14,PENDING,OTHER,PENDING - OTHER,ABC,A,1
8,2,2019-03-14,CANCEL,OTHER,PENDING - OTHER,ABC,A,1
9,2,2019-03-14,ACTIVE,READY,FULFILLMENT,ABC,A,1


In [369]:
expected_data = ([
    [1, 0, 70, 'PENDING - OTHER', 47],
    [1, 70, 72, 'PENDING - OTHER', 2],
    [1, 72, 72, transform.fulfillment_hierarchy, 0],
    [2, 70, 72, 'NO STATUS CLARITY', 2],
    [2, 72, 72, 'PENDING - OTHER', 0],
    [2, 72, 72, 'PENDING - OTHER', 0],
    [2, 72, 73, 'NO STATUS CLARITY', 1],
    [2, 73, 73, transform.fulfillment_hierarchy, 0]
])

expected_df = (
    pd.DataFrame(test_data, columns = [patient_id, status_date, next_status_date, hierarchy, time_in_status])
    .assign(**{
        status_date : lambda x: (
            pd.to_datetime('2019-01-01', format='%Y-%m-%d') + pd.to_timedelta(x[status_date], unit='d')
        ),
        next_status_date : lambda x: (
            pd.to_datetime('2019-01-01', format='%Y-%m-%d') + pd.to_timedelta(x[next_status_date], unit='d')
        ),
        trans_id : 1,
        pharmacy : 'ABC'
    })    
)

expected_df

ValueError: Invalid ISO 8601 Duration format - PENDING

In [361]:
# Apply transform to test dataframe

test_output = (
    test_df
    .pipe(pj)
    .pipe(ttff))

test_output

,pmcTransactionId,pmcPatientId,pharmName,ttffAccruBucket,startDateLong,statusDateLong,minShipDateLong,brand,startDateTs,statusDateTs,timeToFillDays
0,1,1,ABC,PENDING - OTHER,1546300800,1552348800,1552521600,A,2019-01-01,2019-03-12,47
1,1,1,ABC,PENDING - OTHER,1552348800,1552521600,1552521600,A,2019-03-12,2019-03-14,2
2,1,1,ABC,FULFILLMENT,1552521600,1552521600,1552521600,A,2019-03-14,2019-03-14,0
3,1,2,ABC,NO STATUS CLARITY,1552348800,1552521600,1552608000,A,2019-03-12,2019-03-14,2
4,1,2,ABC,PENDING - OTHER,1552521600,1552521600,1552608000,A,2019-03-14,2019-03-14,0
5,1,2,ABC,PENDING - OTHER,1552521600,1552521600,1552608000,A,2019-03-14,2019-03-14,0
6,1,2,ABC,FULFILLMENT,1552521600,1552608000,1552608000,A,2019-03-14,2019-03-15,1
7,1,2,ABC,FULFILLMENT,1552608000,1552608000,1552608000,A,2019-03-15,2019-03-15,0


In [ ]:
# Check that results match expectations

test5 = (
    test_output
    .
    .assign(passfail = lambda x: np.where(
        (x[transform.hierarchy] == x['expected_hierarchy']) | (x[transform.hierarchy].isnull() & x['expected_hierarchy'].isnull()),
        True,
        False
    ))
    .passfail
    .all()
)

test5

In [ ]:
# FINAL TEST: Did all 5 tests pass?

test1 & test2 & test3 & test4 & test5

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()